# Exemplu regresie liniară

Urmărim pașii exemplelor din documentația oficială, cu scopul de a înțelege și apoi aplica metodele pentru rezolvarea altor probleme.
<br>
https://spark.apache.org/docs/latest/ml-classification-regression.html#linear-regression
<br>
https://spark.apache.org/docs/latest/api/java/index.html?org/apache/spark/ml/regression/LinearRegression.html

In [ ]:
from pyspark.sql import SparkSession

In [ ]:
spark = SparkSession.builder.appName('lr_example').getOrCreate()

In [ ]:
from pyspark.ml.regression import LinearRegression

In [ ]:
# Încărcarea datelor de antrenare
training = spark.read.format("libsvm").option("numFeatures","10").load("sample_linear_regression_data.txt")

Observatie: "libsvm" este un format de date, structurat astfel:
`<label> <index1>:<value1> <index2>:<value2> `
<br>
Observatie: Dataset-uri libsvm: https://www.csie.ntu.edu.tw/~cjlin/libsvmtools/datasets/

In [ ]:
training.show()

Acesta este formatul așteptat de Spark, alcătuit din 2 coloane (label, features). 

Coloana "label" trebuie să aibă o valoare numerică, fie o valoare numerică de regresie, fie o valoare numerică ce corespunde unui grup dintr-o clasificare.

Coloana "features" conține un vector având valorile caracteristicilor corespunzătoare acelui rând. De obicei, se ajunge la combinarea diferitelor coloane cu caracteristici într-o singură coloană "features" cu ajutorul transformărilor.

In [ ]:
# Acestea sunt valorile implicite pentru featuresCol, labelCol, predictionCol
lr = LinearRegression(featuresCol='features', labelCol='label', predictionCol='prediction')

# Putem utiliza și alți parametri - de exemplu, 
#pentru regularizare (vezi https://static1.squarespace.com/static/5ff2adbe3fe4fe33db902812/t/6062a083acbfe82c7195b27d/1617076404560/ISLR%2BSeventh%2BPrinting.pdf)

In [ ]:
# Antrenați modelul cu datele "training"
lrModel = lr.fit(training)

In [ ]:
# Afișați coeficienții și intercepția pentru regresia liniară
print("Coefficients: {}".format(str(lrModel.coefficients))) 
print('\n')
print("Intercept:{}".format(str(lrModel.intercept)))

Observație: Există un atribut (`summary`) ce conține mai multe informații.

In [ ]:
# Sumarizarea modelului pentru setul de training și afișarea unor metrici
trainingSummary = lrModel.summary

Analizați documentația și afișați câteva exemple de informații (ce reprezintă fiecare?):

In [ ]:
trainingSummary.residuals.show()
print("RMSE: {}".format(trainingSummary.rootMeanSquaredError))
print("r2: {}".format(trainingSummary.r2))

## Diviziuni Train/Test

Până în acest punct nu am separat setul de date într-un subset de training și unul de test, ci am antrenat pe toate datele - lucru pe care, în general, dorim să îl evităm.
De reținut: nu se poate face o evaluare corectă a unui model analizând performanța lui pe datele pe care a fost antrenat!

În Spark DataFrame există metoda `randomSplit` pentru divizarea aleatoare a datelor. Utilizăm această metodă:

In [ ]:
all_data = spark.read.format("libsvm").option("numFeatures","10").load("sample_linear_regression_data.txt")

In [ ]:
# Procentele de divizare pentru seturile de training/test sunt transmise ca listă.
# În general, sunt utilizate procentele 70%/30% sau 60%/40%, în funcție de volumul de date disponibil și de cât de echilibrate sunt acestea.
train_data,test_data = all_data.randomSplit([0.7,0.3])

In [ ]:
train_data.show()

In [ ]:
test_data.show()

In [ ]:
##Datele de test, fără etichete
unlabeled_data = test_data.select('features')

In [ ]:
unlabeled_data.show()

Vom antrena doar pe baza setului train_data

In [ ]:
#Antrenăm doar pe train_data
correct_model = lr.fit(train_data)

Putem obține un obiect "summary" folosind metoda "evaluate":

In [ ]:
test_results = correct_model.evaluate(test_data)

In [ ]:
test_results.residuals.show()
print("RMSE: {}".format(test_results.rootMeanSquaredError))

Dorim să testăm modelul pe datele neetichetate - în primul rând, acesta este scopul construirii modelului. 
Putem realiza această testare cu ajutorul unei alte metode - `transform()`. Această metodă a fost, de fapt, apelată și în cadrul lui `evaluate()`.

In [ ]:
predictions = correct_model.transform(unlabeled_data)

In [ ]:
predictions.show()

Ce concluzii se pot trage asupra predicției obținute?